In [15]:
# %load vector_add.py
# improt package
import tvm
from tvm import te # te stands for tensor expression
import time
import timeit
import numpy as np
from matplotlib import pyplot as plt
from IPython import display
try:
    import torch 
    print("\"import torch\" success...")
except:
    pass


"import torch" success...


In [16]:
def get_abc(shape, constructor = None):
    """ return random a, b and empty c with the same shape
    """
    np.random.seed(0)
    a = np.random.normal(size = shape).astype(np.float32)
    b = np.random.normal(size = shape).astype(np.float32)
    c = np.empty(shape = shape, dtype = np.float32)
    if constructor is not None:
        a, b, c = [constructor(i) for i in (a, b, c)]
    return a, b, c

# implemention with numpy
n = 100
a, b, c = get_abc(n, None)
c = a + b
def vector_add_normal(n, a, b):
    d = np.empty_like(a)
    for i in range(n):
        d[i] = a[i] + b[i]
    return d
d = vector_add_normal(n, a, b)
np.testing.assert_equal(c, d)

In [17]:
# defining the tvm computation
def vector_add(n):
    """TVM expressoion for vector add"""
    A = te.placeholder((n,), name = 'a')
    B = te.placeholder((n,), name = 'b')
    C = te.compute(A.shape, lambda i : A[i] + B[i], name = 'c')
    return A, B, C

A, B, C = vector_add(n)
print(type(A), type(C))
print(A.dtype,  A.shape)
print(type(A.op), type(C.op))
print(A.op.__class__.__bases__[0])

# create a schedule: execute plan
s = te.create_schedule(C.op)
print(type(s), type(s[C.op]))
print(tvm.lower(s, [A, B, C], simple_mode = True))

<class 'tvm.te.tensor.Tensor'> <class 'tvm.te.tensor.Tensor'>
float32 [100]
<class 'tvm.te.tensor.PlaceholderOp'> <class 'tvm.te.tensor.ComputeOp'>
<class 'tvm.te.tensor.Operation'>
<class 'tvm.te.schedule.Schedule'> <class 'tvm.te.schedule.Stage'>
@main = primfn(a_1: handle, b_1: handle, c_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {a: Buffer(a_2: Pointer(float32), float32, [100], []),
             b: Buffer(b_2: Pointer(float32), float32, [100], []),
             c: Buffer(c_2: Pointer(float32), float32, [100], [])}
  buffer_map = {a_1: a, b_1: b, c_1: c}
  preflattened_buffer_map = {a_1: a_3: Buffer(a_2, float32, [100], []), b_1: b_3: Buffer(b_2, float32, [100], []), c_1: c_3: Buffer(c_2, float32, [100], [])} {
  for (i: int32, 0, 100) {
    c[i] = (a[i] + b[i])
  }
}




In [24]:
# compilation and execution
mod = tvm.build(s, [A, B, C])
print(type(mod))
"""tvm.ndarray.NDarray ---> np.ndarray (.asnumpy())
"""
a, b, c = get_abc(n, tvm.nd.array)
mod(a, b, c)
np.testing.assert_equal(a.asnumpy() + b.asnumpy(), c.asnumpy())

<class 'tvm.driver.build_module.OperatorModule'>
